In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

img_size = 50

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=5)
        self.fc1 = nn.Linear(128*2*2, 512)
        self.fc2 = nn.Linear(512, 2)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2,2))
        x = F.max_pool2d(F.relu(self.conv3(x)), (2,2))
        x = x.view(-1, 128*2*2)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        x = F.softmax(x, dim=1)
        return x

def evaluate_model():
    net = Net()
    net.load_state_dict(torch.load('saved_model.pth'))
    net.eval()
    
    X_test = np.load("melanoma_X_test.npy")
    y_test = np.load("melanoma_y_test.npy")
    
    test_X = torch.Tensor(X_test)
    test_X = test_X / 255
    test_y = torch.Tensor(y_test)
    
    correct = 0
    total = 0
    
    with torch.no_grad():
        for i in range(len(test_X)):
            output = net(test_X[i].view(-1, 1, img_size, img_size))[0]
            guess = "B" if output[0] >= output[1] else "M"
            real_label = test_y[i]
            real_class = "B" if real_label[0] >= real_label[1] else "M"
            
            if guess == real_class:
                correct += 1
            total += 1
            
    accuracy = round(correct/total, 3)
    print(f"Accuracy: {accuracy}")
    return accuracy

if __name__ == "__main__":
    evaluate_model()

/var/folders/br/8_rtb5dd4ng41b3b4_j0zst80000gn/T/ipykernel_59213/1676218649.py:29: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load('saved_model.

Accuracy: 0.879
